## Setup Dependencies and Environments

### Import pre-installed libraries

In [1]:
import pystac_client
import planetary_computer
import rioxarray
import shapely.geometry
import leafmap
import ipywidgets
import folium

### Get the bbox I want

In [2]:
m = leafmap.Map(center=[39.01841680996714, -77.49839544296265], zoom = 15, basemap = "Google Maps")
m

Map(center=[39.01841680996714, -77.49839544296265], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [3]:
bbox1 = [m.west, m.south, m.east, m.north]
print("{}: {}".format(m.center, m.zoom))

[39.01841680996714, -77.49839544296265]: 15.0


### Install dependencies

In [11]:
#!pip install overturemaps
#!pip install palettable
#!pip install lonboard

In [12]:
import numpy as np
import overturemaps
from matplotlib.colors import LogNorm
from palettable.colorbrewer.sequential import Oranges_9

from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_continuous_cmap

In [13]:
table = overturemaps.record_batch_reader("building", bbox1).read_all()

# Temporarily required as of Lonboard 0.8 to avoid a Lonboard bug
table = table.combine_chunks()

In [20]:
table.num_rows

0

Let's color the buildings by their heights. First, convert the "height" column to a numpy array, then replace any null values with 1

In [14]:
heights = table["height"].to_numpy()
heights = np.nan_to_num(heights, nan=1)

Building heights tend to scale non-linearly. That is, the vast majority of buildings are relatively low, but a few buildings are very tall. So that the low buildings aren't completely washed out, we'll use matplotlib's LogNorm to normalize these values according to a log scale.

In [16]:
layer = PolygonLayer(
    # Select only a few attribute columns from the table
    table=table.select(["id", "height", "geometry", "names"]),
    extruded=True,
    get_elevation=heights,
)

ValueError: Geometry type combination is not supported ([])